In [67]:
import sys
import os

# Setup path for custom modules
project_root = os.path.abspath("..")
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)
if project_root not in sys.path:
    sys.path.append(project_root)

# Safe imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, BooleanType
from pyspark.sql.window import Window
from pyspark.ml.feature import *

In [2]:
spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[*]") \
    .config("spark.driver.memory", "16G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.5.3") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/11 14:10:08 WARN Utils: Your hostname, SSMRS3-048E9600 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/11 14:10:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/pipe/.ivy2/cache
The jars for the packages stored in: /home/pipe/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e7f7aaf8-6de0-4ff2-b6bf-9a86a900f8e5;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.5.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in ce

In [3]:
from src.models.fine_tuning import * 
from src.models.inference import apply_inference_to_dataframe
from src.data.NLP_preprocessing import PreprocessingPipeline

2025-04-11 14:10:17.779162: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-11 14:10:17.788721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744373417.799199   30482 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744373417.802346   30482 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744373417.810969   30482 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
sc = spark.sparkContext
sc.addPyFile("../src.zip")

In [42]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [5]:
#spark = sparknlp.start()


In [4]:
df_csv_analyst = spark.read.option("header", True) \
    .option("multiLine", True) \
    .option("quote", '"') \
    .option("escape", '"') \
    .option("mode", "DROPMALFORMED") \
    .csv("../data/raw/analyst_ratings_processed.csv")

In [13]:
df_csv_analyst.sort(asc("date")).show(20)

+-------+--------------------+--------------------+-----+
|     id|               title|                date|stock|
+-------+--------------------+--------------------+-----+
| 883755|How Treasuries an...|2009-02-14 14:02:...|  NAV|
| 522587|Update on the Lux...|2009-04-27 14:39:...|   FT|
|1396488|Update on the Lux...|2009-04-27 14:39:...|    Y|
|   1834|Going Against the...|2009-04-29 08:48:...|    A|
|  68387|Charles Sizemore ...|2009-05-22 14:28:...|   AM|
| 561526|JVA perks to 39% ...|2009-05-27 03:32:...| GMCR|
| 430148|JVA perks to 39% ...|2009-05-27 03:32:...|  EPS|
| 714847|JVA perks to 39% ...|2009-05-27 03:32:...|  JVA|
| 714846|MRM a $15-$20+ st...|2009-05-27 21:35:...|  JVA|
| 430147|MRM a $15-$20+ st...|2009-05-27 21:35:...|  EPS|
|1081067|MRM a $15-$20+ st...|2009-05-27 21:35:...|   RF|
| 430146|In $7.60 UTA - Ne...|2009-05-29 07:47:...|  EPS|
|1048384|In $7.60 UTA - Ne...|2009-05-29 07:47:...|    Q|
| 365056|Saturday super-tr...|2009-05-30 09:32:...|  DIT|
| 430145|Satur

In [5]:
preproc = PreprocessingPipeline(spark)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]

25/04/11 13:34:02 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/04/11 13:34:03 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
Download done! Loading the resource.
[OK!]


In [7]:
df_nlp_analyst = preproc.run(df_csv_analyst, text_col="title")

In [8]:
df_nlp_analyst.write.mode("overwrite").parquet("../data/processed/processed_news_analyst.parquet")

In [5]:
nlp_analyst = spark.read.parquet("../data/processed/processed_news_analyst.parquet")

In [10]:
nlp_analyst.show(10)

+---+--------------------+--------------------+-----+--------------------+--------------------+
| id|            raw_text|                date|stock|     finished_tokens|          final_text|
+---+--------------------+--------------------+-----+--------------------+--------------------+
|  0|Stocks That Hit 5...|2020-06-05 10:30:...|    A|[stock, hit, 52we...|stock hit 52week ...|
|  1|Stocks That Hit 5...|2020-06-03 10:45:...|    A|[stock, hit, 52we...|stock hit 52week ...|
|  2|71 Biggest Movers...|2020-05-26 04:30:...|    A|[71, big, mover, ...| 71 big mover friday|
|  3|46 Stocks Moving ...|2020-05-22 12:45:...|    A|[46, stock, move,...|46 stock move fri...|
|  4|B of A Securities...|2020-05-22 11:38:...|    A|[b, security, mai...|b security mainta...|
|  5|CFRA Maintains Ho...|2020-05-22 11:23:...|    A|[cfra, maintain, ...|cfra maintain hol...|
|  6|UBS Maintains Neu...|2020-05-22 09:36:...|    A|[ubs, maintain, n...|ubs maintain neut...|
|  7|Agilent Technolog...|2020-05-22 09:

In [6]:
df_analyst_sentiment = apply_inference_to_dataframe(nlp_analyst, text_column="final_text")

In [7]:
df_analyst_sentiment.write.partitionBy("stock").mode("overwrite").parquet("../data/processed/sentiment_analyst.parquet")

2025-04-11 14:10:46.978565: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-11 14:10:46.999420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744373447.022300   31450 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744373447.029595   31450 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744373447.054624   31450 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [8]:
SA_analyst = spark.read.parquet("../data/processed/sentiment_analyst.parquet")

In [11]:
SA_analyst.sort(desc("predicted_sentiment")).show(20)

+-------+--------------------+--------------------+--------------------+--------------------+-------------------+-----+
|     id|            raw_text|                date|     finished_tokens|          final_text|predicted_sentiment|stock|
+-------+--------------------+--------------------+--------------------+--------------------+-------------------+-----+
|1329386|Emerging Companie...|2020-03-02 09:03:...|[emerge, company,...|emerge company po...|            LABEL_2| VSLR|
|1358434|10 Ways to Build ...|2011-05-22 18:00:...|[10, way, build, ...|10 way build defe...|            LABEL_2|  WIP|
|1120937|Unheralded $1B ET...|2012-07-20 06:35:...|[unheralded, 1b, ...|unheralded 1b etf...|            LABEL_2| SCPB|
|1047206|More Tech ETFs Th...|2012-02-27 14:51:...|[tech, etfs, dont...|tech etfs dont ne...|            LABEL_2|  PXQ|
|1373624|WVFC Trading Resu...|2015-03-25 10:53:...|[wvfc, trade, res...|wvfc trade resume...|            LABEL_2| WVFC|
|1379591|This Dead Money S...|2013-03-25

In [16]:
SA_analyst.select("stock").distinct().count()


6192

In [17]:
SA_analyst_final = SA_analyst.withColumn(
    "sentiment_numeric",
    when(col("predicted_sentiment") == "LABEL_0", 0)
    .when(col("predicted_sentiment") == "LABEL_1", 1)
    .when(col("predicted_sentiment") == "LABEL_2", 2)
    .otherwise(-1)
)

In [35]:

# If you want to keep the full timestamp (if later you need intraday ordering)
SA_analyst_final = SA_analyst_final.withColumn("news_timestamp", to_timestamp("date", "yyyy-MM-dd HH:mm:ss"))


In [36]:
SA_analyst_final = SA_analyst_final.withColumn("news_date", to_date("date", "yyyy-MM-dd"))

In [40]:
SA_analyst_final.describe

<bound method DataFrame.describe of DataFrame[id: string, raw_text: string, date: string, finished_tokens: array<string>, final_text: string, predicted_sentiment: string, stock: string, sentiment_numeric: int, news_timestamp: timestamp, news_date: date]>

In [43]:
SA_analyst_final.show(20)

+------+--------------------+--------------------+--------------------+--------------------+-------------------+-----+-----------------+-------------------+----------+
|    id|            raw_text|                date|     finished_tokens|          final_text|predicted_sentiment|stock|sentiment_numeric|     news_timestamp| news_date|
+------+--------------------+--------------------+--------------------+--------------------+-------------------+-----+-----------------+-------------------+----------+
|850715|Shares of several...|2020-06-11 10:22:...|[share, several, ...|share several hea...|            LABEL_0|  MRK|                0|2020-06-11 10:22:00|2020-06-11|
|850716|Johnson & Johnson...|2020-06-11 00:16:...|[johnson, johnson...|johnson johnson s...|            LABEL_0|  MRK|                0|2020-06-11 00:16:00|2020-06-11|
|850717|The Daily Biotech...|2020-06-10 07:30:...|[daily, biotech, ...|daily biotech pul...|            LABEL_0|  MRK|                0|2020-06-10 07:30:00|2020

In [45]:
df_news_agg = SA_analyst_final.groupBy("stock", "news_date") \
    .agg(avg("sentiment_numeric").alias("avg_sentiment"))

In [47]:
df_news_agg.show(20)

+-----+----------+-------------+
|stock| news_date|avg_sentiment|
+-----+----------+-------------+
|  MRK|2019-10-01|          0.0|
|  MRK|2014-11-21|          0.0|
|  MRK|2014-10-08|          0.0|
|  MRK|2014-05-02|          0.0|
|  MRK|2010-02-25|          0.0|
| NFLX|2019-11-09|          0.0|
| NFLX|2019-08-29|          0.0|
| NFLX|2018-08-02|          0.0|
| NFLX|2018-03-11|          0.0|
| NFLX|2018-02-02|          0.0|
| NFLX|2017-12-11|          0.0|
|  JNJ|2019-10-14|          0.0|
|  JNJ|2019-09-27|          0.0|
|  JNJ|2019-06-19|          0.0|
|  JNJ|2018-10-19|          0.0|
|  JNJ|2017-11-06|          0.0|
|  JNJ|2015-01-08|          0.0|
|  JNJ|2014-09-30|          0.0|
|  JNJ|2014-08-22|          0.0|
|  JNJ|2014-01-23|          0.0|
+-----+----------+-------------+
only showing top 20 rows



In [57]:
# Create a new column 'effective_date' that corresponds to the next day (i.e., news_date + 1 day)
df_news_agg = df_news_agg.withColumn("effective_date", date_add(col("news_date"), 1))


In [58]:
df_news_agg.show(20)

+-----+----------+-------------+----------+--------------+
|stock| news_date|avg_sentiment| join_date|effective_date|
+-----+----------+-------------+----------+--------------+
|  MRK|2019-10-01|          0.0|2019-10-02|    2019-10-02|
|  MRK|2014-11-21|          0.0|2014-11-22|    2014-11-22|
|  MRK|2014-10-08|          0.0|2014-10-09|    2014-10-09|
|  MRK|2014-05-02|          0.0|2014-05-03|    2014-05-03|
|  MRK|2010-02-25|          0.0|2010-02-26|    2010-02-26|
| NFLX|2019-11-09|          0.0|2019-11-10|    2019-11-10|
| NFLX|2019-08-29|          0.0|2019-08-30|    2019-08-30|
| NFLX|2018-08-02|          0.0|2018-08-03|    2018-08-03|
| NFLX|2018-03-11|          0.0|2018-03-12|    2018-03-12|
| NFLX|2018-02-02|          0.0|2018-02-03|    2018-02-03|
| NFLX|2017-12-11|          0.0|2017-12-12|    2017-12-12|
|  JNJ|2019-10-14|          0.0|2019-10-15|    2019-10-15|
|  JNJ|2019-09-27|          0.0|2019-09-28|    2019-09-28|
|  JNJ|2019-06-19|          0.0|2019-06-20|    2019-06-2

In [19]:
df_prices = spark.read \
    .option("header", "true") \
    .csv("../data/raw/price_data/*.csv") \
    .withColumn("file_path", input_file_name())

In [20]:
df_prices = df_prices.withColumn("ticker",
    regexp_extract(col("file_path"), r"([^/\\]+)_historical\.csv$", 1)
)

In [25]:
df_prices_clean = df_prices.filter(col("Date").isNotNull())

In [38]:
df_prices_clean.sort(desc("Date")).show(20)

+----------+------------------+------------------+------------------+------------------+---------+--------------------+------+
|      Date|             Close|              High|               Low|              Open|   Volume|           file_path|ticker|
+----------+------------------+------------------+------------------+------------------+---------+--------------------+------+
|2020-06-10| 35.38862228393555|37.386511182506446| 35.36174594293514| 37.37755354138716|  2091100|file:///home/pipe...|   TAP|
|2020-06-10|  48.5067253112793|50.072545171702956|48.077389161497884|50.047291048585954|  2498900|file:///home/pipe...|   QSR|
|2020-06-10|15.153594017028809|16.112681923785726|15.128573751550041|16.104341835292804|  4586100|file:///home/pipe...|   UNM|
|2020-06-10|103.48999786376953|104.63999938964844|102.16000366210938| 103.4000015258789|  1415900|file:///home/pipe...|  KEYS|
|2020-06-10| 7.216312885284424| 7.819087288088239| 7.216312885284424| 7.776638226833336|  2643400|file:///home/

In [26]:
df_prices_clean.sort(asc("Date")).show(20)

+----------+------------------+------------------+------------------+------------------+-------+--------------------+------+
|      Date|             Close|              High|               Low|              Open| Volume|           file_path|ticker|
+----------+------------------+------------------+------------------+------------------+-------+--------------------+------+
|2009-04-27|1.3759037256240845| 1.396287464122779|1.3623145666249548|1.3623145666249548|  77500|file:///home/pipe...|    FT|
|2009-04-28|1.3820761442184448| 1.423229734914669|1.3752172124357407|1.3752172124357407| 100200|file:///home/pipe...|    FT|
|2009-04-29|11.505555152893066|11.607656843311771|10.918471921856835|10.982284946628768|5516508|file:///home/pipe...|     A|
|2009-04-29| 1.392364263534546|1.4129410559773448|1.3820759490780512|1.3992231943488123|  57600|file:///home/pipe...|    FT|
|2009-04-30|11.652327537536621|12.086259583684875| 11.55022668035704| 11.55022668035704|4977998|file:///home/pipe...|     A|


In [31]:
df_prices_clean = df_prices_clean.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))

In [32]:
df_prices_clean.describe

<bound method DataFrame.describe of DataFrame[Date: date, Close: string, High: string, Low: string, Open: string, Volume: string, file_path: string, ticker: string]>

In [51]:

"""
window function allows to perform calculations across a set (or window) of rows related to the current row. In other words, every group of rows sharing the same ticker will be treated independently.

"""

w = Window.partitionBy("ticker").orderBy("Date")  


In [52]:
"""
The lag function is a window function that allows you to access a previous row's value from the current row's partition. In this case, it takes the "Close" value (the closing price) from the previous row for the same ticker.

over(w) applies the window specification w (i.e., partitioned by ticker and ordered by Date) to the lag function.
"""

df_prices_clean = df_prices_clean.withColumn("prev_close", lag("Close").over(w))


#### Calculation of daily percentage change 

In [53]:
df_prices_clean = df_prices_clean.withColumn("pct_change", 
                                            when(col("prev_close").isNull(), lit(0.0))
                                            .otherwise((col("Close") - col("prev_close")) / col("prev_close")))

In [ ]:
df_prices_clean = df_prices_clean.drop("prev_close")

In [54]:
df_prices_clean.show(20)

+----------+-----------------+------------------+------------------+------------------+-------+--------------------+------+-----------------+--------------------+
|      Date|            Close|              High|               Low|              Open| Volume|           file_path|ticker|       prev_close|          pct_change|
+----------+-----------------+------------------+------------------+------------------+-------+--------------------+------+-----------------+--------------------+
|2012-04-13|8.568169593811035| 8.746518853159165| 8.501288710142255| 8.731656119590928| 786200|file:///home/pipe...|  ACCO|             NULL|                 0.0|
|2012-04-16|8.753949165344238| 8.902572951683151| 8.486426349934197| 8.620188111986275| 725500|file:///home/pipe...|  ACCO|8.568169593811035|0.021682527347193913|
|2012-04-17|8.456703186035156| 8.709364024763426| 8.456703186035156| 8.627620395354722|1815100|file:///home/pipe...|  ACCO|8.753949165344238|-0.03395564375514...|
|2012-04-18|8.41211509

In [59]:
df_prices_clean = df_prices_clean.withColumn(
    "movement_label",
    when(col("pct_change") > 0, "up")
    .when(col("pct_change") < 0, "down")
    .otherwise("neutral")
)

We use lag("Close").over(w) to compute the previous day’s closing price, and then compute the percent change. Then we define a target variable (label) for the daily price movement.

In [60]:
df_prices_clean.show(20)

+----------+-----------------+------------------+------------------+------------------+-------+--------------------+------+-----------------+--------------------+--------------+
|      Date|            Close|              High|               Low|              Open| Volume|           file_path|ticker|       prev_close|          pct_change|movement_label|
+----------+-----------------+------------------+------------------+------------------+-------+--------------------+------+-----------------+--------------------+--------------+
|2012-04-13|8.568169593811035| 8.746518853159165| 8.501288710142255| 8.731656119590928| 786200|file:///home/pipe...|  ACCO|             NULL|                 0.0|       neutral|
|2012-04-16|8.753949165344238| 8.902572951683151| 8.486426349934197| 8.620188111986275| 725500|file:///home/pipe...|  ACCO|8.568169593811035|0.021682527347193913|            up|
|2012-04-17|8.456703186035156| 8.709364024763426| 8.456703186035156| 8.627620395354722|1815100|file:///home/pi

Now we join both df, this join aligns each day’s price movement with the aggregated sentiment of the previous day’s news. We use a left join so that if a price day has no corresponding news, we still keep the record (filling the news features with default values). For example, set the default "neutral" sentiment at 1.0 (if you encoded neutral as 1) 

In [61]:
df_joined = df_prices_clean.join(
    df_news_agg,
    (df_prices_clean.ticker == df_news_agg.stock) &
    (df_prices_clean.Date == df_news_agg.effective_date),
    how="left"  # Use left join to keep all price records
)

In [63]:
df_final = df_joined.withColumn(
    "movement_numeric",
    when(col("movement_label") == "up", 2)
    .when(col("movement_label") == "neutral", 1)
    .when(col("movement_label") == "down", 0)
    .otherwise(1)
)

In [64]:
df_final.show(20)

+----------+------------------+------------------+------------------+------------------+---------+--------------------+------+------------------+--------------------+--------------+-----+----------+-------------+----------+--------------+----------------+
|      Date|             Close|              High|               Low|              Open|   Volume|           file_path|ticker|        prev_close|          pct_change|movement_label|stock| news_date|avg_sentiment| join_date|effective_date|movement_numeric|
+----------+------------------+------------------+------------------+------------------+---------+--------------------+------+------------------+--------------------+--------------+-----+----------+-------------+----------+--------------+----------------+
|2013-10-03|              10.0|10.010000228881836| 9.800000190734863|              10.0|    72400|file:///home/pipe...|  AAOI| 9.970000267028809|0.003009000217422...|            up| NULL|      NULL|         NULL|      NULL|         

When there is no sentiment available (i.e. no news for that stock on that day), we impute the average sentiment as “neutral” (1). With this we interpret that in that specific day there wasn't any sentiment towards that stock, this might bias the model but for lack of time is the selected approach 

In [65]:
df_final = df_final.fillna({"avg_sentiment": 0})

In [66]:
df_final.show(20)

+----------+------------------+------------------+------------------+------------------+---------+--------------------+------+------------------+--------------------+--------------+-----+----------+-------------+----------+--------------+----------------+
|      Date|             Close|              High|               Low|              Open|   Volume|           file_path|ticker|        prev_close|          pct_change|movement_label|stock| news_date|avg_sentiment| join_date|effective_date|movement_numeric|
+----------+------------------+------------------+------------------+------------------+---------+--------------------+------+------------------+--------------------+--------------+-----+----------+-------------+----------+--------------+----------------+
|2013-10-03|              10.0|10.010000228881836| 9.800000190734863|              10.0|    72400|file:///home/pipe...|  AAOI| 9.970000267028809|0.003009000217422...|            up| NULL|      NULL|          0.0|      NULL|         

# ML models implementation and comparison 

In [68]:
# Define the feature columns. 
feature_cols = ["pct_change", "avg_sentiment"]

In [69]:
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features") #to combine features into a single column 
df_features = assembler.transform(df_final)

In [70]:
indexer = StringIndexer(inputCol="movement_label", outputCol="label")
df_features = indexer.fit(df_features).transform(df_features)

#### Data split

In [71]:
train_df, test_df = df_features.randomSplit([0.8, 0.2], seed=42)

# Logistic Regression

In [82]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [83]:
# Scale the features using StandardScaler
#    We turn on centering (withMean=True) and scaling (withStd=True).
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withMean=True,
    withStd=True
)

In [85]:
# Create the Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100)

# Fit the model
lr_model = lr.fit(train_df)

# Make predictions
lr_predictions = lr_model.transform(test_df)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
lr_f1 = evaluator.evaluate(lr_predictions)

print("Logistic Regression F1 score: {:.2f}%".format(lr_f1 * 100))

25/04/11 18:59:10 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
25/04/11 18:59:10 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
25/04/11 18:59:10 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25
25/04/11 18:59:10 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.5
25/04/11 18:59:10 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.25
25/04/11 18:59:10 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.125
25/04/11 18:59:10 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.0625
25/04/11 18:59:11 ERROR StrongWolfeLineSearch: Encountered bad values in function evaluation. Decreasing step size to 0.03125
25/04/11 18:5

Logistic Regression F1 score: 100.00%


In [76]:
print("Coefficients matrix:\n " + str(lr_model.coefficientMatrix))
print("Intercepts: " + str(lr_model.interceptVector))

Coefficients matrix:
 DenseMatrix([[ 2.48760694e+08, -2.90275116e+02],
             [-2.48759446e+08,  1.40006751e+02],
             [-1.24842701e+03,  1.50268365e+02]])
Intercepts: [-4.502981337633845,-3.7929765935139024,8.295957931147747]


# Random Forest

In [75]:
from pyspark.ml.classification import RandomForestClassifier

In [77]:
# Create the Random Forest classifier model. You can tweak hyperparameters like numTrees, maxDepth, etc.
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=50, maxDepth=5)

# Fit the model
rf_model = rf.fit(train_df)

# Make predictions
rf_predictions = rf_model.transform(test_df)


In [78]:
# Evaluate the model
rf_f1 = evaluator.evaluate(rf_predictions)
print("Random Forest F1: {:.2f}%".format(rf_f1 * 100))

# Optionally inspect feature importance:
feature_importances = rf_model.featureImportances
print("Feature Importances: " + str(feature_importances))

Random Forest F1: 99.99%
Feature Importances: (2,[0],[1.0])
